<a href="https://colab.research.google.com/github/kgautam2103/zoom_participant_list_public/blob/main/zoom_participant_list_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
import requests
import random
import json
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import gspread
import re
from gspread_dataframe import set_with_dataframe
from pandas import json_normalize
from urllib.parse import urlencode
!pip install python-Levenshtein
!pip install fuzzywuzzy
from fuzzywuzzy import process


In [ ]:
report_date= input("enter the report date like 2021-07-11 : ")
move2side_input=input('move2side needed ? Enter true or false: ').lower()
#report_date = '2021-07-18'
apac_report_datetime = str(report_date)+'T01'
naol_report_datetime = str(report_date)+'T13'
meetingID = '9181716151'
base_url = 'https://api.zoom.us/v2/'
past_meetings = base_url+'/past_meetings/'


#to get result for the last held meeting
report_meetings = base_url+'report/meetings/'
#get_meeting_url = report_meetings+meetingID

get_meeting_url = past_meetings+meetingID+'/instances'
get_user_url = base_url+'users/'
apac_uid=''
naol_uid=''
page_size ='300'
apac_participants_list = []
naol_participants_list = []
apac_next_page_token = ''
naol_next_page_token = ''
apac_participants_df=pd.DataFrame()
naol_participants_df=pd.DataFrame()


current_ts = datetime.now().strftime('%Y%m%d%H%M%S')
output_sheet = current_ts+'_participants_list_'+report_date
sh = gc.create(output_sheet)

final_output_sheet = sh.add_worksheet('final_combined',rows="5000", cols="10")
apac_output_sheet = sh.add_worksheet('APAC',rows="500", cols="10")
naol_output_sheet = sh.add_worksheet('NAOL',rows="500", cols="10")
rolling3week_output_sheet = sh.add_worksheet('3weekRollingWeekend',rows="5000", cols="10")
move2side_output_sheet = sh.add_worksheet('move2side',rows="500", cols="10")
updated_regular_output_sheet = sh.add_worksheet('updated_regular_sideroom',rows="1000", cols="10")
apac_poll1_sheet = sh.add_worksheet('APAC_poll1',rows="500", cols="10")
apac_poll2_sheet = sh.add_worksheet('APAC_poll2',rows="500", cols="10")
naol_poll1_sheet = sh.add_worksheet('NAOL_poll1',rows="500", cols="10")
naol_poll2_sheet = sh.add_worksheet('NAOL_poll2',rows="500", cols="10")

dummy_sheet = sh.get_worksheet(0)
sh.del_worksheet(dummy_sheet)

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1WhACIU6F7i9VwDxXoqzx6GEwutQI5HpIYl4v6H49-7Q/edit#gid=0')

input_sheet = wb.worksheet('sahajyogis')
inputdata = input_sheet.get_all_values()
inputdata_df = pd.DataFrame(inputdata, columns=['id','name','email'])
inputdata_df = inputdata_df.drop(inputdata_df.index[0])

rolling_3week_sheet = wb.worksheet('Rolling3Weekend')
rolling_3week_data = rolling_3week_sheet.get_all_values()
rolling_3week_df = pd.DataFrame(rolling_3week_data, columns=['event','event_date','id','name', 'user_email', 'join_time', 'leave_time', 'duration'])
if rolling_3week_df.shape[0] > 0:
  rolling_3week_df = rolling_3week_df.drop(rolling_3week_df.index[0])

weekend_archive_sheet = wb.worksheet('WeekendArchive')
weekend_archive_data = weekend_archive_sheet.get_all_values()
weekend_archive_df = pd.DataFrame(weekend_archive_data, columns=['event','event_date','id','name', 'user_email', 'join_time', 'leave_time', 'duration'])
if weekend_archive_df.shape[0] > 0:
  weekend_archive_df = weekend_archive_df.drop(weekend_archive_df.index[0])

Sideroom_sheet = wb.worksheet('regular_sideroom')
sideroom_data = Sideroom_sheet.get_all_values()
sideroom_data_df = pd.DataFrame(sideroom_data, columns=['name'])
sideroom_data_df = sideroom_data_df.drop(sideroom_data_df.index[0])

move2side_sheet = wb.worksheet('move2side')
updated_sideroom_sheet = wb.worksheet('updated_sideroom')
month_3_archive_frequency_sheet = wb.worksheet('3month_archive_frequency')

report_datetime = datetime.strptime(report_date, '%Y-%m-%d')
report_datetime_3week_before = report_datetime-timedelta(days=15)
report_datetime_3month_before = report_datetime-timedelta(days=90)

ref_id_list = list(filter(None, inputdata_df['id'].tolist()))
ref_name_list = list(filter(None, inputdata_df['name'].tolist()))
ref_name_list = [i.lower() for i in ref_name_list]
ref_email_list = list(filter(None, inputdata_df['email'].tolist()))


#enter the zoom account auth token
auth_token = ''



authorization = 'Bearer ' + auth_token
auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

meeting_id_response = requests.get(get_meeting_url,headers=auth_headers)


if meeting_id_response.status_code == 200:

  for x in meeting_id_response.json()['meetings']:
    if apac_report_datetime in str(x['start_time']):
      apac_uid = str(x['uuid'])
      print(x)
    if naol_report_datetime in str(x['start_time']):
      naol_uid = str(x['uuid'])
      print(x)



print('apac_id : '+apac_uid)
print('naol_id : '+naol_uid)

if apac_uid != "":
  encoded_apac_uid = urlencode({'id':apac_uid})
  double_encoded_apac_id = urlencode({'id':encoded_apac_uid[3:]})
  apac_uid = double_encoded_apac_id[3:]
  print('apac_id_encoded : '+apac_uid)
  apac_poll_response_url = base_url+'report/meetings/'+apac_uid+'/polls'
  get_apac_participants_url = report_meetings+apac_uid+'/participants?page_size='+page_size
  apac_participants_response = requests.get(get_apac_participants_url,headers=auth_headers)
  #print(apac_participants_response.json())
  apac_participants_list = apac_participants_response.json()['participants']
  apac_next_page_token = apac_participants_response.json()['next_page_token']
  while apac_next_page_token != "":
    new_get_apac_participants_url = get_apac_participants_url+'&next_page_token='+apac_next_page_token
    new_apac_participants_response = requests.get(new_get_apac_participants_url,headers=auth_headers)
    apac_next_page_token =  new_apac_participants_response.json()['next_page_token']
    apac_participants_list.extend(new_apac_participants_response.json()['participants'])

  apac_poll_response = requests.get(apac_poll_response_url,headers = auth_headers)
  apac_response_list = apac_poll_response.json()['questions']
  apac_poll_answer_df = json_normalize(apac_response_list,
                                record_path=['question_details'],
                                meta=['name', 'email',],
                                errors='ignore')

  apac_poll1_df = apac_poll_answer_df[apac_poll_answer_df.polling_id=='l1gX0McLRc6mxffk37nnIw']
  if apac_poll1_df.shape[0] > 0:
    apac_poll1_df  = apac_poll1_df.fillna("")
    apac_poll1_df = apac_poll1_df.reset_index()
    apac_poll1_df = apac_poll1_df.drop(columns=['question','polling_id','date_time','email','index'])
    apac_poll1_df = apac_poll1_df[['name','answer']]
    set_with_dataframe(apac_poll1_sheet, apac_poll1_df)

  apac_poll2_df = apac_poll_answer_df[apac_poll_answer_df.polling_id=='mlE7n7f3TQ-qgXoF--MKPQ']
  if apac_poll2_df.shape[0] > 0:
    apac_poll2_df  = apac_poll2_df.fillna("")
    apac_poll2_df['question'] = apac_poll2_df['question'].apply(lambda x: re.sub('^[\.\d\s]*','',x,flags=re.IGNORECASE))
    apac_poll2_df.drop_duplicates(subset=['name','question'],inplace=True)
    apac_poll2_df = apac_poll2_df.pivot(index="name", columns="question", values="answer")
    apac_poll2_df.reset_index(inplace=True)
    apac_poll2_df[['answer1','answer2','answer3']] = apac_poll2_df[['Will you be joining us next weekend?',
                                          'Which of the following mid-week followup sessions are convenenient for you? We look forward to see you',
                                          'How was your thoughtless state during meditation?']]
    apac_poll2_df=apac_poll2_df.drop(columns=['How was your thoughtless state during meditation?',
                                'Will you be joining us next weekend?',
                                      'Which of the following mid-week followup sessions are convenenient for you? We look forward to see you'])
    set_with_dataframe(apac_poll2_sheet, apac_poll2_df) 



if naol_uid != "":
  encoded_naol_uid = urlencode({'id':naol_uid})
  double_encoded_naol_id = urlencode({'id':encoded_naol_uid[3:]})
  naol_uid = double_encoded_naol_id[3:]
  print('naol_id_encoded : '+naol_uid)
  naol_poll_response_url = base_url+'report/meetings/'+naol_uid+'/polls'
  get_naol_participants_url = report_meetings+naol_uid+'/participants?page_size='+page_size
  naol_participants_response = requests.get(get_naol_participants_url,headers=auth_headers)
  #print(naol_participants_response.json()['next_page_token'])
  naol_participants_list = naol_participants_response.json()['participants']
  naol_next_page_token = naol_participants_response.json()['next_page_token']
  while naol_next_page_token != "":
    new_get_naol_participants_url = get_naol_participants_url+'&next_page_token='+naol_next_page_token
    new_naol_participants_response = requests.get(new_get_naol_participants_url,headers=auth_headers)
    naol_next_page_token =  new_naol_participants_response.json()['next_page_token']
    naol_participants_list.extend(new_naol_participants_response.json()['participants'])

  naol_poll_response = requests.get(naol_poll_response_url,headers = auth_headers)
  naol_response_list = naol_poll_response.json()['questions']
  naol_poll_answer_df = json_normalize(naol_response_list,
                                record_path=['question_details'],
                                meta=['name', 'email',],
                                errors='ignore')

  naol_poll1_df = naol_poll_answer_df[naol_poll_answer_df.polling_id=='l1gX0McLRc6mxffk37nnIw']
  if naol_poll1_df.shape[0] > 0:
    naol_poll1_df  = naol_poll1_df.fillna("")
    naol_poll1_df = naol_poll1_df.reset_index()
    naol_poll1_df = naol_poll1_df.drop(columns=['question','polling_id','date_time','email','index'])
    naol_poll1_df = naol_poll1_df[['name','answer']]
    set_with_dataframe(naol_poll1_sheet, naol_poll1_df)

  naol_poll2_df = naol_poll_answer_df[naol_poll_answer_df.polling_id=='mlE7n7f3TQ-qgXoF--MKPQ']
  if naol_poll2_df.shape[0] > 0:
    naol_poll2_df  = naol_poll2_df.fillna("")
    naol_poll2_df['question'] = naol_poll2_df['question'].apply(lambda x: re.sub('^[\.\d\s]*','',x,flags=re.IGNORECASE))
    naol_poll2_df.drop_duplicates(subset=['name','question'],inplace=True)
    naol_poll2_df = naol_poll2_df.pivot(index="name", columns="question", values="answer")
    naol_poll2_df.reset_index(inplace=True)
    naol_poll2_df[['answer1','answer2','answer3']] = naol_poll2_df[['Will you be joining us next weekend?',
                                          'Which of the following mid-week followup sessions are convenenient for you? We look forward to see you',
                                          'How was your thoughtless state during meditation?']]
    naol_poll2_df=naol_poll2_df.drop(columns=['How was your thoughtless state during meditation?',
                                'Will you be joining us next weekend?',
                                      'Which of the following mid-week followup sessions are convenenient for you? We look forward to see you'])
    set_with_dataframe(naol_poll2_sheet, naol_poll2_df) 

print(len(apac_participants_list))
print(len(naol_participants_list))


if len(apac_participants_list)>0:
  apac_participants_df = pd.DataFrame(apac_participants_list)
  apac_participants_df = apac_participants_df.drop(columns=['attentiveness_score','failover','customer_key'])
  apac_participants_df = apac_participants_df[apac_participants_df.duration>600]
  apac_participants_df['id'] = apac_participants_df['id'].apply(lambda x: str(random.randint(1,99999999999)) if x=="" else x)
  apac_participants_df = apac_participants_df.drop_duplicates(subset=['id'])
  apac_participants_df['name'] = apac_participants_df['name'].apply(lambda x: re.sub('^sy[\.\s_-]*','',x,flags=re.IGNORECASE))\
  .apply(lambda x: re.sub('[\.\s_-]*sy$','',x,flags=re.IGNORECASE))
  apac_participants_df = apac_participants_df.drop_duplicates(subset=['name'])
  apac_participants_df = apac_participants_df[~apac_participants_df.id.isin(ref_id_list)]
  apac_participants_df = apac_participants_df[~(apac_participants_df.name).str.lower().isin(ref_name_list)]
  apac_participants_df = apac_participants_df[~apac_participants_df.user_email.isin(ref_email_list)]
  apac_participants_df['event'] = 'APAC'
  apac_participants_df['event_date'] = report_datetime
  apac_participants_df = apac_participants_df[['event','event_date','id','name','user_email', 'join_time', 'leave_time', 'duration']]
  set_with_dataframe(apac_output_sheet, apac_participants_df)
  

if len(naol_participants_list)>0:
  naol_participants_df = pd.DataFrame(naol_participants_list)
  naol_participants_df = naol_participants_df.drop(columns=['attentiveness_score','failover','customer_key'])
  naol_participants_df = naol_participants_df[naol_participants_df.duration>600]
  naol_participants_df['id'] = naol_participants_df['id'].apply(lambda x: str(random.randint(1,99999999999)) if x=="" else x)
  naol_participants_df = naol_participants_df.drop_duplicates(subset=['id'])
  naol_participants_df['name'] = naol_participants_df['name'].apply(lambda x: re.sub('^sy[\.\s_-]+','',x,flags=re.IGNORECASE))\
  .apply(lambda x: re.sub('[\.\s_-]+sy$','',x,flags=re.IGNORECASE))
  naol_participants_df = naol_participants_df.drop_duplicates(subset=['name'])
  naol_participants_df = naol_participants_df[~naol_participants_df.id.isin(ref_id_list)]
  naol_participants_df = naol_participants_df[~(naol_participants_df.name).str.lower().isin(ref_name_list)]
  naol_participants_df = naol_participants_df[~naol_participants_df.user_email.isin(ref_email_list)]
  naol_participants_df['event'] = 'NAOL'
  naol_participants_df['event_date'] = report_datetime
  naol_participants_df = naol_participants_df[['event','event_date','id','name','user_email', 'join_time', 'leave_time', 'duration']]
  set_with_dataframe(naol_output_sheet, naol_participants_df)


#columns=['event','event_date','id','name', 'user_email', 'join_time', 'leave_time', 'duration']
final_combined_df = apac_participants_df.append(naol_participants_df)
set_with_dataframe(final_output_sheet, final_combined_df)

if move2side_input == 'true':
    
  rolling_3week_df_combined = rolling_3week_df.append(final_combined_df)
  weekend_archive_df_final = weekend_archive_df.append(final_combined_df)
  rolling_3week_df_combined['event_date'] = pd.to_datetime(rolling_3week_df_combined['event_date'])
  rolling_3week_df_final = rolling_3week_df_combined[rolling_3week_df_combined.event_date > report_datetime_3week_before]
  rolling_3week_df_final['name']=rolling_3week_df_final['name'].str.lower()
  rolling_3week_df_final.drop_duplicates(subset=['event','event_date','name'],inplace=True)
  weekend_archive_df_final['event_date'] = pd.to_datetime(weekend_archive_df_final['event_date'])
  weekend_archive_df_final['name']=weekend_archive_df_final['name'].str.lower()
  
  weekend_archive_sheet.clear()
  rolling_3week_sheet.clear()
  set_with_dataframe(rolling3week_output_sheet, rolling_3week_df_final)
  set_with_dataframe(weekend_archive_sheet,weekend_archive_df_final)
  set_with_dataframe(rolling_3week_sheet, rolling_3week_df_final)

  weekend_archive_df_final_filtered_3m = weekend_archive_df_final[weekend_archive_df_final.event_date > report_datetime_3month_before]
  weekend_archive_df_final_filtered_3m_frequency = weekend_archive_df_final_filtered_3m.groupby(['name']).agg(count=('name',np.count_nonzero))
  weekend_archive_df_final_filtered_3m_frequency.reset_index(inplace=True)
  weekend_archive_df_final_filtered_3m_frequency.sort_values(by = ['count'],ascending=False,inplace=True)
  set_with_dataframe(month_3_archive_frequency_sheet,weekend_archive_df_final_filtered_3m_frequency)

  weekend_archive_group_df = weekend_archive_df_final.groupby('name')
  archive_max_date = weekend_archive_group_df.agg(event_date=('event_date',np.max))
  archive_max_date.reset_index(inplace=True)
  archive_max_date_final = archive_max_date[archive_max_date.event_date > report_datetime_3month_before]

  archive_group_df_max_date_details = pd.merge(weekend_archive_df_final,archive_max_date_final,how='inner',on=['name','event_date'])
  archive_group_df_max_date_details.drop_duplicates(subset=['name','event_date'],inplace=True)
  archive_group_df_max_date_details.drop(columns=['join_time','leave_time','duration'],inplace=True)
  #archive_group_df_max_date_details

  archive_name_list=archive_group_df_max_date_details['name'].tolist()
  archive_name_list_new = [i for i in archive_name_list if len(i) > 1]
  matched_name_list=[]
  for index,row in sideroom_data_df.iterrows():
    highest_match = process.extractOne(row['name'],archive_name_list_new)
    matched_name_list.append([row['name'],highest_match[0],highest_match[1]])

  matched_name_df = pd.DataFrame(matched_name_list,columns=['name','matched_names','score'])
  matched_name_df = matched_name_df[matched_name_df.score>90]
  matched_name_df_final = pd.DataFrame(matched_name_df['matched_names'])
  matched_name_df_final.rename(columns={'matched_names':'name'},inplace=True)
  matched_name_df_final_details = pd.merge(archive_group_df_max_date_details,matched_name_df_final,how='inner',on='name')

  matched_name_df_final_details['event_date'] = matched_name_df_final_details['event_date'].dt.strftime('%Y-%m-%d')
  matched_name_df_final_details['last_event']=matched_name_df_final_details['event_date'].str.slice(0,10)+" "+matched_name_df_final_details['event']
  matched_name_df_final_details.drop(columns=['event','event_date','id'],inplace=True)

  updated_sideroom_sheet.clear()
  set_with_dataframe(updated_sideroom_sheet, matched_name_df_final_details)
  set_with_dataframe(updated_regular_output_sheet, matched_name_df_final_details)


  group_name_df = rolling_3week_df_final.groupby(['name'])
  count_name = group_name_df.agg(count=('name',np.count_nonzero))
  max_date = group_name_df.agg(latest_date=('event_date',np.max) )
  max_date.reset_index(inplace=True)
  count_name.reset_index(inplace=True)
  final_max_count = pd.merge(count_name,max_date,on='name')
  final_max_count_filtered = final_max_count[final_max_count['count']>1]

  regular_matched_name_list=matched_name_df_final_details['name'].tolist()
  regular_matched_name_list_new = [i for i in regular_matched_name_list if len(i) > 1]
  move2side_name_list=[]
  for index,row in final_max_count_filtered.iterrows():
    highest_match_move2side = process.extractOne(row['name'],regular_matched_name_list_new)
    move2side_name_list.append([row['name'],highest_match_move2side[0],highest_match_move2side[1]])

  move2side_name_df = pd.DataFrame(move2side_name_list,columns=['name','matched_names','score'])
  move2side_name_df = move2side_name_df[move2side_name_df.score>90]
  move2side_name_df_final = pd.DataFrame(move2side_name_df['matched_names'])
  move2side_name_df_final_list = move2side_name_df_final['matched_names'].tolist()
  final_max_count_filtered = final_max_count_filtered[~final_max_count_filtered.name.isin(move2side_name_df_final_list)]
  final_max_count_filtered.drop(columns=['count'],inplace=True)

  email_name_ref_df = weekend_archive_df_final[['name','user_email']]
  email_name_ref_df.drop_duplicates()
  nan_value = float("NaN")
  email_name_ref_df.replace("", nan_value, inplace=True)
  email_name_ref_df.dropna(inplace=True)

  move2side_final_df_with_email = pd.merge(final_max_count_filtered,email_name_ref_df,how='left',on='name')
  move2side_final_df_with_email.drop_duplicates(inplace=True)

  move2side_sheet.clear()
  set_with_dataframe(move2side_sheet, move2side_final_df_with_email)
  set_with_dataframe(move2side_output_sheet,move2side_final_df_with_email)
